# Carga dos dados

In [15]:
import pandas as pd

In [16]:
df_most_relevants = pd.read_csv('../../data/llm_juris_tcu/eval_most_relevants.csv')
df_most_relevants

,QUERY_ID,RANK,DOC_ID,ENGINE,QUERY_TEXT,DOC_TEXT,LLM_SCORE,LLM_REASONING
0,1,1,21064,(BM25|STS)+Reranker,técnica e preço,Admite-se a preponderância da técnica sobre o ...,3.0,O enunciado aborda diretamente a relação entre...
1,72,2,21064,(BM25|STS)+Reranker,ponderação técnica e preço,Admite-se a preponderância da técnica sobre o ...,3.0,"O enunciado aborda diretamente a consulta, tra..."
2,1,2,42166,(BM25|STS)+Reranker,técnica e preço,Diante da complexidade e da natureza do objeto...,3.0,"O enunciado aborda diretamente a consulta, men..."
3,111,8,42166,(BM25|STS)+Reranker,Qual é a modalidade de licitação utilizada par...,Diante da complexidade e da natureza do objeto...,2.0,O enunciado aborda a modalidade de licitação '...
4,1,3,13702,(BM25|STS)+Reranker,técnica e preço,"Nas licitações do tipo técnica e preço, é irre...",3.0,O enunciado aborda diretamente a consulta sobr...
...,...,...,...,...,...,...,...,...
1495,100,1,17892,(BM25|STS)+Reranker,assistência médica a servidores,A contratação de entidade para prestação de se...,2.0,O enunciado aborda a contratação de entidade p...
1496,150,1,17892,(BM25|STS)+Reranker,Qual é a regra geral para a contratação de ent...,A contratação de entidade para prestação de se...,3.0,"O enunciado responde diretamente à pergunta, m..."
1497,106,7,17267,(BM25|STS)+Reranker,O pregão é adequado para concessões de uso de ...,No âmbito das concessões realizadas pela Agênc...,1.0,O enunciado está relacionado ao tópico de conc...
1498,150,8,91180,(BM25|STS)+Reranker,Qual é a regra geral para a contratação de ent...,É possível a contratação da execução indireta ...,1.0,O enunciado de jurisprudência aborda a contrat...


# Estatísticas por query

In [43]:
df_stats = df_most_relevants.groupby(['QUERY_ID', 'LLM_SCORE']).count()['DOC_ID']
df_stats = df_stats.reset_index()
df_stats.columns = ['QUERY_ID', 'LLM_SCORE', 'COUNT']
df_stats = df_stats.astype('Int32')
df_stats['COUNT'] = df_stats['COUNT'].astype('Int32')
df_stats = df_stats.pivot(index='QUERY_ID', columns='LLM_SCORE', values='COUNT')
df_stats = df_stats.fillna(0)
df_stats.columns = ['SCORE_0', 'SCORE_1', 'SCORE_2', 'SCORE_3']
df_stats['TOTAL'] = df_stats['SCORE_0'] + df_stats['SCORE_1'] + df_stats['SCORE_2'] + df_stats['SCORE_3']
df_stats

,SCORE_0,SCORE_1,SCORE_2,SCORE_3,TOTAL
QUERY_ID,,,,,
1,0,0,0,10,10
2,3,2,2,3,10
3,0,0,4,6,10
4,0,0,0,10,10
5,0,0,0,10,10
...,...,...,...,...,...
146,0,0,9,1,10
147,0,2,7,1,10
148,0,1,6,3,10


# Estatísticas gerais e por grupos

In [44]:
df_stats.describe()

,SCORE_0,SCORE_1,SCORE_2,SCORE_3,TOTAL
count,150.0,150.0,150.0,150.0,150.0
mean,0.193333,1.48,3.24,5.086667,10.0
std,0.748122,2.119643,2.558523,3.24593,0.0
min,0.0,0.0,0.0,0.0,10.0
25%,0.0,0.0,1.0,2.0,10.0
50%,0.0,0.0,3.0,5.0,10.0
75%,0.0,2.0,5.0,8.0,10.0
max,5.0,9.0,9.0,10.0,10.0


In [67]:
df_stats_groups = pd.DataFrame(columns = ['SCORE_0', 'SCORE_1','SCORE_2', 'SCORE_3', 'TOTAL'])
df_stats_groups.loc['GROUP_1'] = df_stats.loc[1:50].mean()
df_stats_groups.loc['GROUP_2'] = df_stats.loc[51:100].mean()
df_stats_groups.loc['GROUP_3'] = df_stats.loc[101:150].mean()
df_stats_groups.loc['ALL'] = df_stats.mean()
df_stats_groups

,SCORE_0,SCORE_1,SCORE_2,SCORE_3,TOTAL
GROUP_1,0.120000,0.56,2.86,6.460000,10.0
GROUP_2,0.180000,2.20,3.38,4.240000,10.0
GROUP_3,0.280000,1.68,3.48,4.560000,10.0
ALL,0.193333,1.48,3.24,5.086667,10.0


# Gravação das estatísticas

In [69]:
df_queries = pd.concat([pd.read_csv('../../data/juris_tcu/query1.csv', sep=';', index_col='QUERY_ID'),
                        pd.read_csv('../../data/juris_tcu/query2.csv', sep=';', index_col='QUERY_ID'),
                        pd.read_csv('../../data/juris_tcu/query3.csv', sep=';', index_col='QUERY_ID')])
df_queries

,QUERY_TEXT
QUERY_ID,
1,técnica e preço
2,restos a pagar
3,aditivo a contrato
4,adesão a ata de registro de preços
5,sobrepreço e superfaturamento
...,...
146,É possível a inclusão de serviços já previstos...
147,O princípio da legalidade estrita pode ser afa...
148,Quais fontes devem ser prioritárias na pesquis...


In [72]:
df_stats = df_stats.merge(df_queries, on='QUERY_ID')

In [73]:
df_stats.to_csv('../../data/llm_juris_tcu/eval_statistics.csv')
df_stats_groups.to_csv('../../data/llm_juris_tcu/eval_statistics_groups.csv')